# 2. Decision Variables for Optimization Models

This notebook is part of a step-by-step guide to creating an optimization model. We continue our 7-step procedure:

1. ✓ Import Pyomo and Define the Sets
2. ➡️ Define the Decision Variables
3. Define the Parameters
4. Define the Expressions
5. Define the Objective Function
6. Define the Constraints
7. Solve the Model

In this notebook, we will explore different types of decision variables and how to define them in Pyomo. Decision variables are the unknowns in our optimization problem that we want to determine. They represent quantities that we can control or adjust to achieve our objective.

We will cover:
- Continuous variables (NonNegativeReals)
- Binary variables
- Integer variables
- Variables with bounds
- Indexed variables using sets
- Common expressions using variables

## Setup: Importing Pyomo and Creating a Model

As in the previous notebook, we start by importing Pyomo and creating a concrete model:

In [1]:
import pyomo.environ as pyo

# Create a concrete model
m = pyo.ConcreteModel()

## Types of Decision Variables

In optimization problems, we often encounter different types of decision variables. Here are the main types:

1. Continuous Variables:
   - Can take any real value (possibly within bounds)
   - Example: Amount of raw materials to purchase

2. Binary Variables:
   - Can only take values 0 or 1
   - Often used for yes/no decisions
   - Example: Whether to build a facility at a location

3. Integer Variables:
   - Can only take integer values
   - Example: Number of machines to purchase

Let's explore how to create each type in Pyomo.

### 1. Continuous Variables

Let's start with continuous variables. We'll create two sets and some continuous variables:

In [2]:
# Create sets for products and machines
m.I = pyo.Set(initialize=['P1', 'P2', 'P3'])  # Products
m.J = pyo.Set(initialize=['M1', 'M2'])       # Machines

# Production amount for each product (non-negative continuous)
m.x = pyo.Var(m.I, domain=pyo.NonNegativeReals)

# Production time allocated to each product on each machine
m.y = pyo.Var(m.I, m.J, domain=pyo.NonNegativeReals)

Note here that we have two sets, $I$ and $J$. The first set $I$ is the set of products, and the second set $J$ is the set of machines. The decision variable $x$ is indexed by the set $I$, and the decision variable $y$ is indexed by the sets $I$ and $J$. That means, we have:

- $x_i$ for $i \in I$. For example, $x_{P1}$ is the production amount for product $P1$.
- $y_{i,j}$ for $i \in I$ and $j \in J$. For example, $y_{P1,M1}$ is the production time allocated to product $P1$ on machine $M1$.

Let's examine what we created:

In [3]:
# Print the sets
print(f"Products: {[i for i in m.I]}")
print(f"Machines: {[j for j in m.J]}")

Products: ['P1', 'P2', 'P3']
Machines: ['M1', 'M2']


In [4]:
# Print the x variables
for i in m.I:
    print(m.x[i])

x[P1]
x[P2]
x[P3]


In [5]:
# Print the t variables
for i in m.I:
    for j in m.J:
        print(m.y[i,j])

y[P1,M1]
y[P1,M2]
y[P2,M1]
y[P2,M2]
y[P3,M1]
y[P3,M2]


Similar with the previous notebook, we can assign initial values to these variables. Let's create a dictionary for the initial values for $x$ and $y$:

In [6]:
init_x = {
    'P1': 100, 'P2': 150, 'P3': 200
}

init_y = {
    ('P1','M1'): 10, ('P1','M2'): 5,
    ('P2','M1'): 15, ('P2','M2'): 10,
    ('P3','M1'): 20, ('P3','M2'): 15
}


We can then initialize the variables with these values:

In [7]:
m = pyo.ConcreteModel()
m.I = pyo.Set(initialize=['P1', 'P2', 'P3'])  # Products
m.J = pyo.Set(initialize=['M1', 'M2'])       # Machines
m.x = pyo.Var(m.I, initialize=init_x)
m.y = pyo.Var(m.I, m.J, initialize=init_y)

Note that it is much easier to assign initial values to the variables using the dictionaries, since we do not need to use lambda functions. Instead, we can directly assign the values to the variables, as long as we have the correct indices. 

We can print the model to see the initial values:

In [8]:
m.pprint()

2 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'P1', 'P2', 'P3'}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'M1', 'M2'}

2 Var Declarations
    x : Size=3, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         P1 :  None :   100 :  None : False : False :  Reals
         P2 :  None :   150 :  None : False : False :  Reals
         P3 :  None :   200 :  None : False : False :  Reals
    y : Size=6, Index=I*J
        Key          : Lower : Value : Upper : Fixed : Stale : Domain
        ('P1', 'M1') :  None :    10 :  None : False : False :  Reals
        ('P1', 'M2') :  None :     5 :  None : False : False :  Reals
        ('P2', 'M1') :  None :    15 :  None : False : False :  Reals
        ('P2', 'M2') :  None :    10 :  None : False : False :  Reals
        ('P3', 'M1') 

Similar as in the previous notebook, we can do summation of the variables using the `sum` with indices.

In [9]:
print("Sum of x for i in I: ", sum(m.x[i] for i in m.I))
print("The value of the sum is: ", sum(m.x[i]() for i in m.I))

Sum of x for i in I:  x[P1] + x[P2] + x[P3]
The value of the sum is:  450


Similarly, for two indices, we can do the following:

In [10]:
print("Sum of y for i in I and j in J: ", sum(m.y[i,j] for i in m.I for j in m.J))
print("The value of the sum is: ", sum(m.y[i,j]() for i in m.I for j in m.J))


Sum of y for i in I and j in J:  y[P1,M1] + y[P1,M2] + y[P2,M1] + y[P2,M2] + y[P3,M1] + y[P3,M2]
The value of the sum is:  75


We can also multiply the variables and sum them up. We can store the result in a new variable.

In [11]:
sum_x_y = sum(m.x[i] * m.y[i,j] for i in m.I for j in m.J)

We can then print and show the value of the sum:

In [12]:
print("The sum product is: ", sum_x_y)
print("The value of the sum product is: ", sum_x_y())


The sum product is:  x[P1]*y[P1,M1] + x[P1]*y[P1,M2] + x[P2]*y[P2,M1] + x[P2]*y[P2,M2] + x[P3]*y[P3,M1] + x[P3]*y[P3,M2]
The value of the sum product is:  12250


### 2. Binary Variables

Binary variables are useful for yes/no decisions. Let's create some binary variables to represent whether we use a machine for a product:

In [13]:
# Initialize the variables
init_z = {
    ('P1','M1'): 1, ('P1','M2'): 0,
    ('P2','M1'): 1, ('P2','M2'): 1,
    ('P3','M1'): 0, ('P3','M2'): 1
}

# Binary variable: 1 if machine j is used for product i, 0 otherwise
m.z = pyo.Var(m.I, m.J, domain=pyo.Binary, initialize=init_z)

We can print the variables to see the initial values:

In [14]:
m.z.pprint()

z : Size=6, Index=I*J
    Key          : Lower : Value : Upper : Fixed : Stale : Domain
    ('P1', 'M1') :     0 :     1 :     1 : False : False : Binary
    ('P1', 'M2') :     0 :     0 :     1 : False : False : Binary
    ('P2', 'M1') :     0 :     1 :     1 : False : False : Binary
    ('P2', 'M2') :     0 :     1 :     1 : False : False : Binary
    ('P3', 'M1') :     0 :     0 :     1 : False : False : Binary
    ('P3', 'M2') :     0 :     1 :     1 : False : False : Binary


`pprint()` is a useful function to print Pyomo model and its components in a readable format.

### 3. Integer Variables

Integer variables are useful when we need whole number solutions. Let's create some integer variables for the number of batches to produce:

In [15]:
# Integer variable: number of batches to produce of each product

# Set up the initial values
init_b = {
    'P1': 10, 'P2': 15, 'P3': 20
}

# Initialize the variables
m.b = pyo.Var(m.I, domain=pyo.NonNegativeIntegers, initialize=init_b)

# Print the variables
m.b.pprint()

b : Size=3, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     P1 :     0 :    10 :  None : False : False : NonNegativeIntegers
     P2 :     0 :    15 :  None : False : False : NonNegativeIntegers
     P3 :     0 :    20 :  None : False : False : NonNegativeIntegers


### 4. Variables with Bounds

We can add bounds to our variables. This is useful when we know the valid ranges for our variables:

In [16]:
# Production amount with minimum and maximum bounds
init_w = {
    'P1': 100, 'P2': 150, 'P3': 200
}

m.w = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize=init_w, bounds=(100,500))

# Print the variables
m.w.pprint()

w : Size=3, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     P1 :   100 :   100 :   500 : False : False : NonNegativeReals
     P2 :   100 :   150 :   500 : False : False : NonNegativeReals
     P3 :   100 :   200 :   500 : False : False : NonNegativeReals


We can also create variables with different bounds for each index:

In [17]:
# Dictionary of bounds for each machine
bounds_v = {
    'M1': (100, 200),
    'M2': (150, 300),
}

# Dictionary for initial values
init_v = {
    'M1': 100, 'M2': 150,
}

# Production amount with machine-specific bounds
m.v = pyo.Var(m.J, domain=pyo.NonNegativeReals, bounds=bounds_v, initialize=init_v)

# Print the variables
m.v.pprint()

v : Size=2, Index=J
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     M1 :   100 :   100 :   200 : False : False : NonNegativeReals
     M2 :   150 :   150 :   300 : False : False : NonNegativeReals


## Common Expressions Using Variables

Now let's look at some common expressions we can create using these variables:

### 1. Sum of Variables

```python
sum_x = sum(m.x[i] for i in m.I)
```

we can also use Pyomo's `summation` function to create the same expression:

```python
sum_x = pyo.summation(m.x)
```


In [18]:
sum_x = pyo.summation(m.x)

print("The sum of x is: ", sum_x)
print("The value of the sum of x is: ", sum_x())

The sum of x is:  x[P1] + x[P2] + x[P3]
The value of the sum of x is:  450


### 2. Product of Variables

We can use Pyomo's built-in `prod` function to create the product of the variables:

```python
prod_x = pyo.prod(m.x[i] for i in m.I)
```

In [19]:
prod_x = pyo.prod(m.x[i] for i in m.I)

print("The product of x is: ", prod_x)
print("The value of the product of x is: ", prod_x())

The product of x is:  x[P1]*x[P2]*x[P3]
The value of the product of x is:  3000000


### 3. Sum of Products of Variables

```python
sum_prod_y_z = sum(m.y[i,j] * m.z[i,j] for i in m.I for j in m.J)
```

or, we can use Pyomo's built-in functions to create these expressions:

```python
sum_prod_y_z = pyo.sum_product(m.y, m.z)
```


In [20]:
sum_prod_y_z = pyo.sum_product(m.y, m.z)

print("The sum of the product of y and z is: ", sum_prod_y_z)
print("The value of the sum of the product of y and z is: ", sum_prod_y_z())

The sum of the product of y and z is:  y[P1,M1]*z[P1,M1] + y[P1,M2]*z[P1,M2] + y[P2,M1]*z[P2,M1] + y[P2,M2]*z[P2,M2] + y[P3,M1]*z[P3,M1] + y[P3,M2]*z[P3,M2]
The value of the sum of the product of y and z is:  50


## Conclusion

In this notebook, we have explored different types of decision variables and how to define them in Pyomo. We have also looked at some common expressions we can create using these variables.

In the next notebook, we will look at how to define the parameters in our optimization model.

## Practice

Now, it's your turn to practice:

1. Create a concrete model $m$ with two sets, $I$ and $J$
   $I = \{1, 2, ..., 10\}$ and $J = \{1, 2, ..., 5\}$.
2. Create a continuous variable $x$ indexed by $I$ with initial values $init_x = \{1, 2, ..., 10\}$.
3. Create a binary variable $y$ indexed by $I$ and $J$ with initial values 0 for all $i \in I$ and $j \in J$.
4. Create a integer variable $z$ indexed by $I$ with initial values 10 with bounds (0, 20).
5. Calculate the sum of $x$ for all $i \in I$ and store the result in a new variable $sum_x$.
6. Calculate the product of $y$ for all $i \in I$ and $j \in J$ and store the result in a new variable $prod_y$.
7. Calculate the sum of the product of $x$ and $z$ for all $i \in I$ and $j \in J$ and store the result in a new variable $sumprodxz$.
8. Calculate $ysum_i = \sum_{j \in J} y_{i,j}$ for all $i \in I$ and store the result in a new variable $ysum_i$.
9. Calculate the sum of the product of $x$ and $ysum_i$ for all $i \in I$ and store the result in a new variable $sumprodxysum$.
10. Calculate the sum of the product of $x, ysum_i, z$ for all $i \in I$ and store the result in a new variable $sumprodxyz$.

In [21]:
# 1. Create a concrete model $m$ with two sets, $I$ and $J$
# Your code should contain at least 3 lines of code.
# 1 line for creating the model, m = ...
# 1 line for creating the set I of m, m.I = ...
# 1 line for creating the set J of m, m.J = ...
### Write your code below ###


In [22]:
# 2. Create a continuous variable $x$ indexed by $I$ with initial values $init_x = \{1, 2, ..., 10\}$.
# Start with initiating init_x as a dictionary with keys as the indices of I and values as the initial values.
# init_x = {1: 1, 2: 2, ...}
# Then, create the variable x of m, m.x = ...
### Write your code below ###


In [23]:
# 3. Create a binary variable $y$ indexed by $I$ and $J$ with initial values 0 for all $i \in I$ and $j \in J$.
### Write your code below ###


In [24]:
# 4. Create a integer variable $z$ indexed by $I$ with initial values 10 with bounds (0, 20).
### Write your code below ###


In [25]:
# 5. Calculate the sum of $x$ for all $i \in I$ and store the result in a new variable $sum_x$.
### Write your code below ###


In [26]:
# 6. Calculate the product of $y$ for all $i \in I$ and $j \in J$ and store the result in a new variable $prod_y$.
### Write your code below ###


In [27]:
# 7. Calculate the sum of the product of $x$ and $z$ for all $i \in I$ and $j \in J$ and store the result in a new variable $sumprodxz$.
### Write your code below ###


In [28]:
# 8. Calculate $ysum_i = \sum_{j \in J} y_{i,j}$ for all $i \in I$ and store the result in a new variable $ysum_i$.
# Use list comprehension, since you will have to iterate over the indices of J for each i in I.
# The result should be a dictionary with keys as the indices of I and values as the sums.
# ysum_i = {i: ... for i in m.I}
### Write your code below ###


In [29]:
# 9. Calculate the sum of the product of $x$ and $ysum_i$ for all $i \in I$ and store the result in a new variable $sumprodxysum$.
### Write your code below ###


In [30]:
# 10. Calculate the sum of the product of $x, ysum_i, z$ for all $i \in I$ and store the result in a new variable $sumprodxyz$.
### Write your code below ###
